## 1. Setting

In [1]:
from config import *
import os
os.chdir(PROJECT_PATH)

import requests
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import DBSCAN
import xml.etree.ElementTree as elemTree

import lib.inputManager as IM
import lib.userManager as UM
import lib.stationManager as SM
import lib.numpyPlus as NPLUS

## 2. Loading

In [2]:
# 로딩할 파일 명 리스트 생성
input_path_list = IM.make_input_path(START_DATE, END_DATE)

# 전체 데이터 로딩 (multi)
usage_df = IM.parallel_load_total_usage_df(input_path_list)
# usage_df = IM.load_total_usage_df(input_path_list)

## 3. Extracting and Analyzing

#### 3.1 Extracting station and user

In [3]:
#정류장 추출
station_df = SM.create_station_df(usage_df)

#유저 목록 생성
user_df = UM.create_user_df(usage_df)

#### 3.2 first station analysis

In [40]:
#정류장 주소 계산 및 컬럼 삽입
station_df = SM.set_station_address_column(station_df)

# 전체 이용자에 대한 정류장별 이용량 분석 수행
orders = [["total",   "geton",  "total_geton"],
          ["total",   "getoff", "total_getoff"]]
station_df = SM.analyze_station_usage(station_df, user_df, usage_df, orders)

# 공항과 항만에 대한 이름 후보(정류장 이름의 부분 문자열) 데이터
airport_candidate_list = ["제주국제공항"]
harbor_candidate_list = ["국제여객터미널", "제주연안여객터미널", "제4부두", "제6부두", "임항로", "제주해양경찰서"]
    
# 공항 정류장을 추출 후 station_df에 속성으로 삽입
airport_station_df = SM.extract_related_stations(station_df, "station_name", airport_candidate_list)
station_df = NPLUS.insert_flag_column(station_df, airport_station_df, id_column = "station_id", result_column = "airport_flag")

# 항만 정류장을 추출 후 station_df에 속성으로 삽입
harbor_station_df = SM.extract_related_stations(station_df, "station_name", harbor_candidate_list)
station_df = NPLUS.insert_flag_column(station_df, harbor_station_df, id_column = "station_id", result_column = "harbor_flag")

# candidate_df는 기존의 station_df에 수작업으로 candidate컬럼 생성 요구
tour_station_candidate_df = pd.read_csv(TOUR_STATION_CANDIDATE_DF_PATH, low_memory=False, encoding = DEFAULT_ENCODING) #, dtype=dtype)
station_df = SM.analyze_tour_station(station_df, tour_station_candidate_df, result_column="tour_station_flag")

100%|██████████████████████████████████████████████████████████████████████████████| 3654/3654 [04:35<00:00, 13.26it/s]


There is no column  'tourist'
create 'tourist' column that the value is False


#### 3.3 User analysis

In [41]:
# 이용 날짜 수 분석
user_df = UM.analyze_usage_date(user_df, usage_df)

# 버스 이용 횟수 분석
user_df = UM.analyze_usage_num(user_df, usage_df)

# 이용 비율 분석
user_df = UM.analyze_usage_ratio(user_df)

# 출발, 도작지 유형 분석
user_df = UM.analyze_first_last_station_type(user_df, usage_df, station_df, "first", "last", "case")

# 이용한 출입정류장(공항) 수 속성 삽입
airport_station_df = station_df[station_df["airport_flag"] == True]
user_df = UM.analyze_visit_count(user_df, usage_df, airport_station_df, result_column="airport_count")

# 이용한 출입정류장(항만) 수 속성 삽입
harbor_station_df = station_df[station_df["harbor_flag"] == True]
user_df = UM.analyze_visit_count(user_df, usage_df, harbor_station_df, result_column="harbor_count")

# 출입정류장 수 속성 통합
user_df["access_count"] = user_df["airport_count"] + user_df["harbor_count"]

# 이용한 관광정류장 수 속성 삽입
tour_station_df = station_df[station_df["tour_station_flag"] == True]
user_df = UM.analyze_visit_count(user_df, usage_df, tour_station_df, result_column="tour_count")

# 관광객 분석
user_df = UM.analyze_and_insert_tourist_column(user_df, column_name = "tourist")

# analyze_station_usage()에 대한 작업 내용 파라미터 셋
orders = [["tourist", "geton",  "tourist_geton_count"],
          ["tourist", "getoff", "tourist_getoff_count"]]

#### 3.4 Second staion analysis

In [42]:
# 관광객 이용자에 대한 정류장별 이용량 분석 수행
station_df = SM.analyze_station_usage(station_df, user_df, usage_df, orders)

## 4. Store

In [5]:
# 정류장 데이터 저장
station_df.to_csv(STATION_DF_PATH, encoding=DEFAULT_ENCODING, index=False)

# 이용자 데이터 저장
user_df.to_csv(USER_DF_PATH, encoding=DEFAULT_ENCODING, index=False)

In [3]:
# station_df = pd.read_csv(STATION_DF_PATH, encoding = DEFAULT_ENCODING)
# user_df = pd.read_csv(USER_DF_PATH, encoding = DEFAULT_ENCODING)